In [3]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bson import ObjectId


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "SNHU1234"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
#print(len(df.to_dict(orient='records')))
#print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

image_filename = 'Grazioso.png' # replaced with the Grazioso image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Center(
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))
    ),  # Include the image tag
    html.Center(html.B(html.H1("Blake's Client-Server Authentication Results - SNHU CS-340"))), #Unique Identifier
    html.Hr(),
    html.Div(
        # Radio Items to select the rescue filter options
        dcc.RadioItems(
            id='filter-type',
            options=[
                # List of options for the radio buttons
                {'label': 'Water Rescue', 'value': 'water'}, # Option label and value
                {'label': 'Mountain/Wilderness Rescue', 'value': 'mount'},
                {'label': 'Disaster Rescue/Individual Tracking', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='RESET', # Default selected value ('RESET' in this case)
            labelStyle={'display': 'inline-block'} # Styling for the labels of the radio buttons
            )),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        
        # interactive features from Module 6
        selected_rows=[0],
        sort_action='native',
        sort_mode="multi",
        filter_action='native',
        page_action='native',
        page_current=0,
        page_size=10,
        style_table={'overflowX': 'auto'},
        style_data_conditional=[
            {'if': {'row_index': 'odd'}, 'backgroundColor': 'rgb(248, 248, 248)'},
            {'if': {'row_index': 'even'}, 'backgroundColor': 'rgb(230, 230, 230)'}
        ],
        style_header={'backgroundColor': 'rgb(230, 230, 230)', 'fontWeight': 'bold'},
        row_selectable='single'
    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

        ),
        html.Div(
            id='map-id',
            className='col s12 m6',
        )
    ])
])
#############################################
# Interaction Between Components / Controller
#############################################
# Radio button filters
@app.callback(
    [Output('datatable-id', 'data'), Output('datatable-id', 'columns')],
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    if filter_type == 'water':
        # Filter criteria for water rescue
        filter_criteria = {
            "animal_type": "Dog",
            "breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26.0, "$lte": 156.0}
        }
    elif filter_type == 'mount':
        # Filter criteria for mountain/wilderness rescue
        filter_criteria = {
            "animal_type": "Dog",
            "breed": {"$in": ["German Shepard", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26.0, "$lte": 156.0}
        }
    elif filter_type == 'disaster':
        # Filter criteria for disaster rescue/individual tracking
        filter_criteria = {
            "animal_type": "Dog",
            "breed": {"$in": ["Doberman Pinscher", "German Shepard", "Golden Retriever", "Bloodhound", "Rottweiler"]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20.0, "$lte": 300.0}
        }
    else:
        filter_criteria = {}  # No filter

    # Fetch data from the database based on the filter criteria
    df = pd.DataFrame.from_records(db.read(filter_criteria))
    df.drop(columns=['_id'], inplace=True)

    # Convert DataFrame data into dictionary format for use in DataTable component
    data = df.to_dict('records')
    columns = [{"name": col_name, "id": col_name, "deletable": False, "selectable": True} for col_name in df.columns]

    return data, columns

# Change background color of selected columns
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    # Check if selected_columns is None (i.e., no selected columns)
    if selected_columns is None:
        # Set default value to an empty list to prevent errors
        selected_columns = []
        
    # Return a list of dictionaries containing style data conditional rules
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")]
)
def update_graphs(viewData):
    try:
        #print("Debug - View Data:", viewData)  # Print the viewData for debugging
        
        dff = pd.DataFrame.from_dict(viewData)
        
        if not dff.empty and 'breed' in dff:
            histogram_fig = px.histogram(dff, x='breed', title='Distribution of Breeds')
            histogram_graph = dcc.Graph(figure=histogram_fig)
            return [histogram_graph]
        else:
            return []  # Return an empty list if no data or no 'breed' column
        
    except Exception as e:
        print("Error:", e)
        return []  # Return an empty list in case of any exceptions
    
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    # Convert the received data to a DataFrame
    dff = pd.DataFrame.from_dict(viewData)

    # Check if viewData or index is None (i.e., no data or no selected row)
    if viewData is None or index is None or not index:
        # Set default values for viewData and index to prevent errors
        viewData = []
        index = [0]

    # Convert the updated viewData to a DataFrame
    dff = pd.DataFrame.from_dict(viewData)

    # Check if the DataFrame is empty (i.e., no data in the DataFrame)
    if dff.empty:
        return []  # Return an empty list if DataFrame is empty

    # Extract the row index from the index list
    row = index[0]

    # Define the map centered at [30.75,-97.48] with zoom level 10
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[30.75, -97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                # Define a marker with tooltip and popup
                # Column 13 and 14 define the grid-coordinates for
                # the map
                # Column 4 defines the breed for the animal
                # Column 9 defines the name of the animal
                dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]],
                          children=[
                              dl.Tooltip(dff.iloc[row, 4]),
                              dl.Popup([
                                  html.H1("Animal Name"),
                                  html.P(dff.iloc[row, 9])
                              ])
                          ])
            ])
    ]


app.run_server(debug=True)


Connection Successful
Dash app running on http://127.0.0.1:12045/
